In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from numpy import zeros, newaxis
from evaluate_paddlepaddle import evaluate
import metrics
import keras
from data_loader import *
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU
from tensorflow.keras.callbacks import EarlyStopping

for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)


2022-07-10 14:20:15.458025: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-10 14:20:15.459237: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-10 14:20:15.460503: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-10 14:20:15.471491: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-10 14:20:15.472597: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from S

In [5]:
# Load dataset
full_data = pd.read_csv("data/wtbdata_245days.csv")
train_data = pd.read_csv("data/train_data.csv")
submission = pd.read_csv("data/sample_submission.csv")
test_data = pd.read_csv("info/target.csv")
full_data = full_data.fillna(method = 'bfill')
train_data = train_data.fillna(method = 'bfill')
full_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4727520 entries, 0 to 4727519
Data columns (total 13 columns):
 #   Column   Dtype  
---  ------   -----  
 0   TurbID   int64  
 1   Day      int64  
 2   Tmstamp  object 
 3   Wspd     float64
 4   Wdir     float64
 5   Etmp     float64
 6   Itmp     float64
 7   Ndir     float64
 8   Pab1     float64
 9   Pab2     float64
 10  Pab3     float64
 11  Prtv     float64
 12  Patv     float64
dtypes: float64(10), int64(2), object(1)
memory usage: 468.9+ MB


In [6]:
# Preprocessing
# Time Stamp to float hour+time
full_data.Tmstamp =full_data.Tmstamp.apply(lambda x: int(x.split(':')[0]) + int(x.split(':')[1]) / 60)

# add cyclical encoded time feature
full_data['cos_time'] = np.cos(2 * np.pi * (full_data.Tmstamp / 24))
full_data['sin_time'] = np.sin(2 * np.pi * (full_data.Tmstamp / 24))


In [77]:
# Split data
TurbID = 1
SEQ_LEN = 2
train_x, train_y = make_train_data_single_turbine(full_data, SEQ_LEN,TurbID)
val_x,   val_y   = make_val_data_single_turbine(full_data, SEQ_LEN, TurbID)
test_x, test_y   = make_test_data_single_turbine(full_data, SEQ_LEN, TurbID)
val_x,   val_y   = val_x.reshape(-1, *train_x.shape[1:]), val_y.reshape(-1, train_y.shape[1])
test_x, test_y   = test_x.reshape(-1, *train_x.shape[1:]),test_y.reshape(-1, train_y.shape[1])
print("Train data(X, y):", train_x.shape, train_y.shape)
print("Val data(X, y):", val_x.shape, val_y.shape)
print("Test data(X, y):", test_x.shape, test_y.shape)

Train data(X, y): (196, 288, 11) (196, 288)
Val data(X, y): (1, 288, 11) (1, 288)
Test data(X, y): (1, 288, 11) (1, 288)


In [78]:
# # Split data single feature
# TurbID = 1
# SEQ_LEN = 5
# train_x, train_y = make_train_data_single_turbine(full_data, SEQ_LEN, TurbID)
# val_x,   val_y   = make_val_data_single_turbine(full_data, SEQ_LEN, TurbID)
# test_x, test_y   = make_test_data_single_turbine(full_data, SEQ_LEN, TurbID)
# train_x,   train_y   = train_x.reshape(-1, *train_x.shape[1:],1), train_y.reshape(-1, train_y.shape[1])
# val_x,   val_y   = val_x.reshape(-1, *train_x.shape[1:]), val_y.reshape(-1, train_y.shape[1])
# test_x, test_y   = test_x.reshape(-1, *train_x.shape[1:]),test_y.reshape(-1, train_y.shape[1])
# submission = submission[submission.TurbID == TurbID]
# test_data = test_data[test_data.TurbID == TurbID]
# print("Train data(X, y):", train_x.shape, train_y.shape)
# print("Val data(X, y):", val_x.shape, val_y.shape)
# print("Test data(X, y):", test_x.shape, test_y.shape)

In [79]:
# Generate dataset
BATCH_SIZE = 128  
train_ds = generate_dataset(train_x, train_y, batch_size=BATCH_SIZE)

In [80]:
# Modeling
model = Sequential([
    GRU(144*SEQ_LEN,input_shape=train_x[0].shape),
    Dense(512, activation='relu'),
    Dense(288, activation='relu')
])
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

In [81]:
# Training
EPOCHS = 1000
model.fit(train_ds, validation_data=[test_x, test_y], epochs=EPOCHS, callbacks=[EarlyStopping(patience=10, restore_best_weights=True)])
model.save(f'saved_model/model_ep{EPOCHS}_b{BATCH_SIZE}_seq{SEQ_LEN}_turb{TurbID}')

Epoch 1/1000
2/2 [==============================] - 3s 464ms/step - loss: 513249.0625 - mae: 479.0134 - val_loss: 399654.0625 - val_mae: 391.7267
Epoch 2/1000
2/2 [==============================] - 0s 62ms/step - loss: 512750.2188 - mae: 478.7903 - val_loss: 399027.1250 - val_mae: 391.5029
Epoch 3/1000
2/2 [==============================] - 0s 63ms/step - loss: 511930.9688 - mae: 478.4236 - val_loss: 398058.6250 - val_mae: 391.1740
Epoch 4/1000
2/2 [==============================] - 0s 61ms/step - loss: 510667.4375 - mae: 477.8577 - val_loss: 396617.9062 - val_mae: 390.7059
Epoch 5/1000
2/2 [==============================] - 0s 61ms/step - loss: 508823.3438 - mae: 477.0331 - val_loss: 394630.5625 - val_mae: 390.0828
Epoch 6/1000
2/2 [==============================] - 0s 61ms/step - loss: 506270.3125 - mae: 475.8968 - val_loss: 392013.3125 - val_mae: 389.2846
Epoch 7/1000
2/2 [==============================] - 0s 61ms/step - loss: 502875.3438 - mae: 474.3974 - val_loss: 388543.4688 - va

INFO:tensorflow:Assets written to: saved_model/model_ep1000_b128_seq2_turb1/assets


INFO:tensorflow:Assets written to: saved_model/model_ep1000_b128_seq2_turb1/assets


In [85]:
submission = pd.read_csv("data/sample_submission.csv")
test_data = pd.read_csv("info/target.csv")
submission = submission[submission.TurbID == TurbID]
test_data = test_data[test_data.TurbID == TurbID]

submission['Patv'] = model.predict(test_x).reshape(-1)
submission.to_csv(f"output/prediction220710_turb{TurbID}.csv", index=False)

1/1 [==============================] - 0s 52ms/step


In [83]:
submission = submission.iloc[:,-1].to_numpy().reshape(1,-1,1)
raw_data = test_data
ground_truth = test_data.iloc[:,-1].to_numpy().reshape(1,-1,1)
submission.shape, ground_truth.shape

((1, 288, 1), (1, 288, 1))

In [84]:
avg_mae, avg_rmse = metrics.turbine_scores(submission, ground_truth, raw_data, 144*2, stride=1)
print('Aprox. Score for Turbine',TurbID,'\n')
print('RMSE: ',avg_rmse*134,'MAE: ',avg_mae*134)
print('\nScore : ', (avg_rmse+avg_mae)*134/2 )

Aprox. Score for Turbine 1 

RMSE:  44.41449331860637 MAE:  39.82125790644625

Score :  42.117875612526305
